In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re

## Web Scrapping IMSSS
### Web Scrapping de la página general por años

In [16]:
base_url = "http://datos.imss.gob.mx"
r = requests.get("http://datos.imss.gob.mx/dataset")
soup = BeautifulSoup(r.content, "html.parser") 
spans = soup.find_all("a", {'data-format':'csv'}, href=True)
links = []
for link in spans:
    href = link.get("href")
    if '/dataset/asg-' in href:
        condicion = int(re.findall(r'\d{4}', href)[0])
        if condicion >= 2018 and condicion < 2023:
            links.append(base_url + href)

### Web Scrapping pt 2 por meses

In [17]:
links_2 = []
for i in range(len(links)):
    r_2 = requests.get(links[i])
    soup_2 = BeautifulSoup(r_2.content, "html.parser")
    spans_2 = soup_2.find_all("a", {'property':'dcat:accessURL'}, href=True)
    for link in spans_2:
        links_2.append(base_url + link.get("href"))

In [18]:
df = []
for i in range(len(links_2)):
    r_3 = requests.get(links_2[i])
    soup_3 = BeautifulSoup(r_3.content, "html.parser")
    spans_3 = soup_3.find_all("a", href=True)
    for link in spans_3:
        if '.csv' in link.get("href"):
            df.append(link.get("href"))

In [ ]:

# Crear un DataFrame vacío para almacenar los datos combinados
df_combined = pd.DataFrame()

for i in range(len(df)):
    # Leer el archivo CSV y especificar el tipo de datos adecuado (si es necesario)
    df_csv = pd.read_csv(df[i], encoding="latin-1", sep="|", dtype={"columna_con_tipo_mezclado": str}, low_memory=False)

    columns = ['rango_uma', 'ta', 'teu', 'tec', 'tpu', 'tpc',
               'ta_sal', 'teu_sal', 'tec_sal', 'tpu_sal', 'tpc_sal', 'masa_sal_ta',
               'masa_sal_teu', 'masa_sal_tec', 'masa_sal_tpu', 'masa_sal_tpc']

    # Usar copy() para evitar SettingWithCopyWarning
    df_sonora = df_csv.loc[(df_csv["sector_economico_1"] == 0) & (df_csv["cve_entidad"] == 26)].copy()

    # Eliminar columnas
    df_sonora.drop(columns, inplace=True, axis=1)

    # Extraer año y mes de la URL
    df_sonora['año'] = re.findall(r'\d{4}', df[i])[0]
    df_sonora['mes'] = re.findall(r'-(\d{2})-', df[i])[0]

    # Filtrar por sector_economico_2
    df_append = df_sonora[df_sonora["sector_economico_2"].isin([1, 2, 4])]

    # Usar pd.concat en lugar de append (sin ignore_index=True)
    df_combined = pd.concat([df_combined, df_append])

In [ ]:
# from google.colab import files
# df_combined.to_csv('df_imss.csv') 
# files.download('df_imss.csv')

In [88]:
df_imss = pd.read_csv('df_imss.csv')

In [89]:
df_imss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629639 entries, 0 to 629638
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          629639 non-null  int64  
 1   cve_delegacion      629639 non-null  int64  
 2   cve_subdelegacion   629639 non-null  int64  
 3   cve_entidad         629639 non-null  int64  
 4   cve_municipio       629639 non-null  object 
 5   sector_economico_1  629639 non-null  float64
 6   sector_economico_2  629639 non-null  float64
 7   sector_economico_4  629639 non-null  float64
 8   tamaño_patron       628822 non-null  object 
 9   sexo                629639 non-null  int64  
 10  rango_edad          629639 non-null  object 
 11  rango_salarial      628979 non-null  object 
 12  asegurados          629639 non-null  int64  
 13  no_trabajadores     629639 non-null  int64  
 14  año                 629639 non-null  int64  
 15  mes                 629639 non-nul

## Web Scraping Agricultura (2018-2022)

In [85]:
url = 'https://datos.sonora.gob.mx/conjuntos-de-datos/mostrar/datos-de-agricultura-sonora/1573'
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")
spans = soup.find_all("a", href=True)

# Lista de años
years = ['2018', '2019', '2020', '2021', '2022']
links_xlsx = []
# Recorre la lista de años y busca enlaces con títulos que coincidan con cada año
for year in years:
    links = soup.find_all("a", {'title': f'Producción agrícola {year}'}, href=True)
    for link in links:
        links_xlsx.append(link['href'])

In [86]:
df_agricultura = pd.DataFrame()
for i in range(len(links_xlsx)):
    df_agricultura = pd.concat([df_agricultura, pd.read_excel(links_xlsx[i])])

In [43]:
df_agricultura.to_csv('df_agricultura.csv')

In [87]:
df_agricultura.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7219 entries, 0 to 1453
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ANO          7219 non-null   int64  
 1   CIERREYAVAN  7219 non-null   object 
 2   CICLO        7219 non-null   int64  
 3   CDDR         7219 non-null   int64  
 4   NDDR         7219 non-null   object 
 5   CMUN         7219 non-null   int64  
 6   NMUN         7219 non-null   object 
 7   CVECUL       7219 non-null   int64  
 8   CULTIVO      7219 non-null   object 
 9   SUPSEM       7219 non-null   float64
 10  SUPCOSE      7219 non-null   float64
 11  SUPSINI      7219 non-null   float64
 12  PRODTON      7219 non-null   float64
 13  RENDMNTO     7219 non-null   float64
 14  PMR          7052 non-null   float64
 15  VALPROD      7219 non-null   float64
dtypes: float64(7), int64(5), object(4)
memory usage: 958.8+ KB


## Web Scraping Pecuaria (2018-2022)

In [67]:
# URL de la página HTML
url = 'https://datos.sonora.gob.mx/conjuntos-de-datos/mostrar/datos-ganaderia-sonora/1581'

# Realiza la solicitud HTTP
r = requests.get(url)

# Parsea el contenido HTML
soup = BeautifulSoup(r.content, "html.parser")

# Lista de años
years = ['2018', '2019', '2020', '2021', '2022']

# Lista para almacenar los enlaces .xlsx
links_xlsx = []

# Expresión regular para buscar enlaces .xlsx
xlsx_pattern = re.compile(r'\.xlsx$')

# Busca todos los enlaces con extensión .xlsx en la página
enlaces = soup.find_all('a', href=xlsx_pattern)

# Recorre los enlaces y verifica si el texto del enlace contiene un año de la lista
for enlace in enlaces:
    texto_enlace = enlace.text
    for year in years:
        if year in texto_enlace:
            links_xlsx.append(enlace['href'])

# Elimina duplicados (si los hay) utilizando set y luego convierte de nuevo a lista
links_xlsx = list(set(links_xlsx))

In [74]:
df_pecuaria = pd.DataFrame()
for i in range(len(links_xlsx)):
    df_pecuaria = pd.concat([df_pecuaria, pd.read_excel(links_xlsx[i])])

In [84]:
df_pecuaria.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1610 entries, 0 to 319
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ANO        1610 non-null   int64  
 1   CIERYAVAN  1610 non-null   object 
 2   CDDR       1610 non-null   int64  
 3   NDDR       1610 non-null   object 
 4   CMUN       1610 non-null   int64  
 5   NMUN       1610 non-null   object 
 6   CVEMES     1610 non-null   int64  
 7   NMES       1610 non-null   object 
 8   CVESPE     1610 non-null   int64  
 9   ESPECIE    1610 non-null   object 
 10  PRODUC     1610 non-null   object 
 11  UMED       1610 non-null   object 
 12  VOLTON     1610 non-null   float64
 13  VALPROD    1610 non-null   float64
dtypes: float64(2), int64(5), object(7)
memory usage: 188.7+ KB


In [90]:
df_pecuaria.to_csv('df_pecuaria.csv')

## Web Scraping Pesca (2018-2022)

In [94]:
# URL de la página HTML
url = 'https://datos.sonora.gob.mx/conjuntos-de-datos/mostrar/datos-pesca-sonora/1582'

# Realiza la solicitud HTTP
r = requests.get(url)

# Parsea el contenido HTML
soup = BeautifulSoup(r.content, "html.parser")

# Lista de años
years = ['2018', '2019', '2020', '2021', '2022']

# Lista para almacenar los enlaces .xlsx
links_xlsx = []

# Expresión regular para buscar enlaces .xlsx
xlsx_pattern = re.compile(r'\.xlsx$')

# Busca todos los enlaces con extensión .xlsx en la página
enlaces = soup.find_all('a', href=xlsx_pattern)

# Recorre los enlaces y verifica si el texto del enlace contiene un año de la lista
for enlace in enlaces:
    texto_enlace = enlace.text
    for year in years:
        if year in texto_enlace:
            links_xlsx.append(enlace['href'])

# Elimina duplicados (si los hay) utilizando set y luego convierte de nuevo a lista
links_xlsx = list(set(links_xlsx))

In [96]:
df_pesca = pd.DataFrame()
for i in range(len(links_xlsx)):
    df_pesca = pd.concat([df_pesca, pd.read_excel(links_xlsx[i])])

In [99]:
df_pesca.ANO.unique()

array([2018, 2019, 2022, 2021, 2020], dtype=int64)

In [97]:
df_pesca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1244 entries, 0 to 245
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ANO         1244 non-null   int64 
 1   CDDR        1244 non-null   int64 
 2   NDDR        1244 non-null   object
 3   CMUN        1244 non-null   int64 
 4   CVEOF       1244 non-null   int64 
 5   NOFICINA    1244 non-null   object
 6   CVOTRAS     1244 non-null   int64 
 7   DOTRAS      1244 non-null   object
 8   CESPE       1244 non-null   int64 
 9   ESPECIE     1244 non-null   object
 10  CVORIGEN    1244 non-null   int64 
 11  ORIGEN      1244 non-null   object
 12  PESODESEMB  1244 non-null   int64 
 13  PVIVO       1244 non-null   int64 
 14  VALOR       1244 non-null   int64 
dtypes: int64(10), object(5)
memory usage: 155.5+ KB


In [98]:
df_pesca.to_csv('df_pesca.csv')